In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('kaggle/input/ukraine-ml-bootcamp-2023/train.csv').to_numpy()
dataset

In [ ]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image
import keras
import cv2
import mediapipe as mp

In [ ]:
# For finding pose cords
mp_pose = mp.solutions.pose

pose_detector = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.0,
    min_tracking_confidence=0.0)

In [ ]:
img = Image.open('kaggle/input/ukraine-ml-bootcamp-2023/images/train_images/1684a6b47dd92daca9b0803d18f1285a.jpg')
img = np.array(img.resize((160, 160)))
plt.imshow(img)

In [ ]:
def image_to_pose_cords(img):
    results = pose_detector.process(np.array(img))
    if results.pose_landmarks is None:
        array = np.zeros((33, 4))
    else:
        a = results.pose_landmarks.ListFields()[0][1]
        array = []
        for val in a:
            array.append([val.x, val.y, val.z, val.visibility])
    
    np.expand_dims(array, -1)
    return array

In [ ]:
test_cords = np.array(image_to_pose_cords(img))

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
    ax.scatter(test_cords[:, 0], test_cords[:, 1], test_cords[:, 2], marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
# Create array with all images
img_data = []
pose_data = []

data_dir = 'kaggle/input/ukraine-ml-bootcamp-2023/images/train_images/'
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

from image_preprocessing import get_train_ds

# ds = get_train_ds(dataset, data_dir, IMG_SIZE, False)

In [ ]:
# img_data = []
# pose_data = []
# for pose, cords in ds:
#     img_data.append(cords)
#     pose_data.append(pose)

In [ ]:
# img_data = np.array(img_data)
# pose_data = np.array(pose_data)
# pose_data_ohe = np.array(tf.one_hot(pose_data, 6))

In [ ]:
# np.save('pose_arrays/pose_data_ohe.npy', pose_data_ohe)
# np.save('pose_arrays/img_data.npy', img_data)

In [ ]:
pose_data_ohe = np.load('pose_arrays/pose_data_ohe.npy')
img_data = np.load('pose_arrays/img_data.npy')
print(img_data.shape, pose_data_ohe.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(img_data, pose_data_ohe, train_size=0.9)
print(len(X_train), len(X_test))

# Model definition

In [ ]:
from keras.layers import Conv1D, Conv2D,MaxPooling2D, Flatten, Dense, Dropout, SimpleRNN
model = keras.Sequential([
    keras.Input((33, 4, 1)),
    Conv2D(64, (4, 4), activation='relu'),
    Conv2D(128, (4, 1), activation='relu'),
    # MaxPooling2D(2, 2),
    # Conv2D(32, (2, 4), activation='relu'),
    # Conv1D(128, (4), activation='relu',),
    # Conv2D(64, (2, 2), activation='relu'),
    # Conv1D(64, (2), activation='relu', input_shape=(33, 4, 1)),
    # MaxPooling2D(2, 2),
    Flatten(),
    # Dense(256, activation='relu', input_shape=(33, 4)),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])

# Training the model

In [ ]:
from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = keras.callbacks.ModelCheckpoint('model.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

In [ ]:
learning_rate = 3e-3
model.compile(loss="categorical_crossentropy", 
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              metrics=["accuracy"])


In [ ]:
history = model.fit(X_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=200,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop, checkpoint])

In [ ]:
# Plot training & validation accuracy, F1 score, and loss values
plt.figure(figsize=(15, 5))

# Plotting Accuracy
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting Loss
plt.subplot(1, 3, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Creating predictions submissions file

In [ ]:
model.load_weights('model.h15')

In [ ]:
# Load sample submission, change values to model forecasts
test_data_dir = 'kaggle/input/ukraine-ml-bootcamp-2023/images/test_images/'

df = pd.read_csv('kaggle/input/ukraine-ml-bootcamp-2023/sample_submission.csv')

for i, (name, pose) in enumerate(df.to_numpy()):
    img_path = os.path.join(test_data_dir, name)
    img = preprocess_path(img_path)
    cords = image_to_pose_cords(img)
    pose_model = model.predict(np.array([cords]), verbose=0)[0]
    if i < 5:
        print(pose_model)
    df.at[i,'class_6'] = np.argmax(pose_model)

In [ ]:
df.to_csv('submission.csv', index=False)